# Models: Neural Network Matrix Factorization

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
BASE_PATH = '../..'
LIB_PATH  = f'{BASE_PATH}/lib'

In [3]:
import sys
sys.path.append(LIB_PATH)

import pandas as pd
from bunch import Bunch

import torch
import pytorch_common
import pytorch_common.util as pu

import model as ml
import service as srv

import util as ut

from sklearn.model_selection import train_test_split

2023-09-02 16:45:04.258986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 16:45:04.840905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-02 16:45:05.464900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-02 16:45:05.480541: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

<Figure size 640x480 with 0 Axes>

## Setup

In [4]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [5]:
import pytorch_common
pytorch_common.__version__

'0.3.2'

In [6]:
torch.__version__

'2.0.1+cu118'

In [7]:
pu.set_device_name('gpu')

pu.get_device(), torch.cuda.is_available()

(device(type='cuda', index=0), True)

In [8]:
ut.set_seed(42)

## Definicion del modelo

In [16]:
params = Bunch({
    'lr'                : 0.001,
    'lr_factor'         : 0.1,
    'lr_patience'       : 8,
    'epochs'            : 15,
    'embedding_size'    : 50,
    'units_per_layer'   : [50, 10, 1],
    'dropout'           : 0.2,
    'n_workers'         : 24,
    'batch_size'        : 512,
    'features_n_values' : [len(it) for it in dataset.features_uniques],
})

model = ml.NNMF(
    params.features_n_values,
    params.embedding_size,
    params.units_per_layer,
    params.dropout
).to(pu.get_device())
model



NNMF(
  (embedding): MultiFeatureEmbedding(
    (embedding): Embedding(19393, 50)
  )
  (mlp): MultiLayerPerceptron(
    (mlp): Sequential(
      (0): Linear(in_features=100, out_features=50, bias=True)
      (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=50, out_features=10, bias=True)
      (5): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
      (8): Linear(in_features=10, out_features=1, bias=True)
    )
  )
  (act): ReLU()
)